In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50 )
import matplotlib.pyplot as plt
import seaborn as sns
import re

sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import pprint
pp = pprint.PrettyPrinter(width=41, compact=True)

from tqdm import tqdm_notebook as tqdm


import warnings
warnings.simplefilter('ignore')

# part 1 - billboard

In [132]:
#https://github.com/guoguo12/billboard-charts
data = pd.read_csv('billboard_original.csv')

In [ ]:
billboard = data[['Performer', 'Song', 'WeekID', 'Week Position', 'Instance', 'Weeks on Chart']]
billboard.columns = ['artist', 'track_name','weekid','week_position', 'instance', 'weeks_on_chart']

In [199]:
billboard.head()

,artist,track_name,weekid,week_position,instance,weeks_on_chart,artist_clean
0,jane child,don't wanna fall in love,2/10/1990,75,1,1,jane child
1,jane child,don't wanna fall in love,2/17/1990,53,1,2,jane child
2,jane child,don't wanna fall in love,2/24/1990,43,1,3,jane child
3,jane child,don't wanna fall in love,3/3/1990,37,1,4,jane child
4,jane child,don't wanna fall in love,3/10/1990,27,1,5,jane child


In [203]:
billboard['weekid'] = pd.to_datetime(billboard['weekid'],format='%m/%d/%Y')
pd.DataFrame(billboard['weekid'].value_counts())[pd.DataFrame(billboard['weekid'].value_counts()).weekid != 100]
print(billboard.weekid.min())
print(billboard.weekid.max())

1958-08-02 00:00:00
2019-06-22 00:00:00


In [152]:
billboard.shape

(317795, 8)

### clean billboard

In [142]:
billboard.artist = billboard.artist.apply(lambda x: x.lower())
billboard.track_name = billboard.track_name.apply(lambda x: x.lower())

In [149]:
billboard['artist_clean'] = billboard.loc[:, 'artist']

In [150]:
#billboard['artist_clean'] = billboard['artist_clean'].apply(lambda x: re.sub(r'(\(feat.*)','', x))
billboard.artist_clean =  billboard.artist_clean.apply(lambda x: x.replace('(','').replace(')','')) 
billboard.artist_clean =  billboard.artist_clean.apply(lambda x: x.replace('[','').replace(']',''))

### featuring

In [151]:
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' featuring ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' featuring: ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' feauring ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' feat. ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' with ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' x ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' starring ')[0]) 
billboard.artist_clean = billboard.artist_clean.apply(lambda x:  x.split(' co-starring ')[0])

### comma

In [156]:
billboard[(billboard.artist_clean == 'diplo, french montana & lil pump')|(billboard.artist_clean == 
                                        'earth, wind & fire')].drop_duplicates(subset='artist')

,artist,track_name,weekid,week_position,instance,weeks_on_chart,artist_clean
823,"earth, wind & fire",mighty mighty,3/16/1974,86,1,2,"earth, wind & fire"
15098,"earth, wind & fire with the emotions",boogie wonderland,5/12/1979,69,1,1,"earth, wind & fire"
252376,"diplo, french montana & lil pump featuring zha...",welcome to the party,6/2/2018,84,1,1,"diplo, french montana & lil pump"


there are some exceptions for artists who do not appear as a solo artist in their own right. in this case, the & means featuring. there may be cases where the matched solo artist is in a collective group but for the sake of simplicitiy I will remove the 'featured' artist.

In [28]:
#originally used: billboard.artist_clean = billboard.artist_clean.apply(lambda x: x if x in meaningful_comma 
#else x.split(',')[0]) but it took too long

meaningful_comma = ['earth, wind & fire', 'ray, goodman & brown', 'crosby, stills & nash', 'peter, paul & mary', 
'mcguinn, clark & hillman',  'emerson, lake & palmer',  'blood, sweat & tears',  'ray, goodman & brown',
'crosby, stills, nash & young',  'artists of then, now & forever', 'hamilton, joe frank & reynolds', 
'tyler, the creator', 'isley, jasper, isley', 'lipps, inc.', 'dino, desi & billy', 'the checkmates, ltd.',
'cotton, lloyd & christian', 'emerson, lake & powell', 'bobby pedrick, jr.', 'dave dee, dozy, beaky, mick and tich']

feat_comma = ['benny blanco, halsey & khalid',  'benny blanco, tainy, selena gomez & j balvin', 
'casper magico, nio garcia, darell, nicky jam, ozuna & bad bunny', 'diplo, french montana & lil pump',
 'farruko, nicki minaj, bad bunny, 21 savage & rvssian', 'jay rock, kendrick lamar, future & james blake',
'joe sherman, his orchestra and chorus',  'kristen bell, agatha lee monn & katie lopez',
 'lunay, daddy yankee & bad bunny', 'ross lynch, grace phipps and jason evigan',
     'sofia carson, cameron boyce, booboo stewart & mitchell hope']

miscellaneous_comma = {'g. dep, p. diddy & black rob':'g. dep', 'ramsey lewis and earth, wind & fire':'ramsey lewis',
'hugh jackman, keala settle, zac efron, zendaya & the greatest showman ensemble':'the greatest showman ensemble',  
'barry devorzon and perry botkin, jr.':'barry devorzon',  "r.l., snoop dogg & lil' kim":'snoop dogg', 
'ramsey lewis and earth, wind & fire':'ramsey lewis', 'tragedy, capone, infinite':'capone'}


sorted_comma = []
main_appears_elsewhere_comma = []
wrong_check_comma = []

for artist in billboard['artist_clean']:
    main_artist = artist.split(',')[0] 
    if ',' in artist:
        
        # comma part of name (from wrong_check_comma)
        if artist in meaningful_comma:
            sorted_comma.append(artist)
            
        # [0] is a legit solo artist aka comma means feat. (from wrong_check_comma)
        elif artist in feat_comma: 
            sorted_comma.append(main_artist)
            
        elif artist in list(miscellaneous_comma.keys()):
            sorted_comma.append(miscellaneous_comma[artist])
        
        # if [0] appears in artists then append [0]
        # check: [0] might not be a legit solo artist
        elif main_artist in billboard.artist_clean.unique(): 
            sorted_comma.append(main_artist)
            main_appears_elsewhere_comma.append(main_artist)
        
        # append the rest
        # check: [0] could be a legit solo artist (when [0] doesnt appear in artists)
        else:
            sorted_comma.append(artist)
            wrong_check_comma.append(artist)
            
    else:
        sorted_comma.append(artist)     

In [ ]:
for artist in billboard['artist_clean']:
    main_artist = artist.split(',')[0] 
    if ',' in artist:
        
        if artist in meaningful_comma:
            sorted_comma.append(artist)

        elif artist in feat_comma: 
            sorted_comma.append(main_artist)

In [30]:
billboard.artist_clean = sorted_comma

### slash

In [31]:
# 'm' was found in main_appears elsewhere which is m/a/r/r/s

In [32]:
# if slash is meaningful
meaningful_slash = ['ac/dc', 'rufus/chaka khan', 'm/a/r/r/s', 'hughes/thrall']

after_slash_real_artist = ['john & yoko/the plastic ono band', 'genius/gza', 'jay-z/linkin park', 
                              "michael zager's moon band/peabo bryson"]

actually_featuring = [ 'marvin hamlisch/"the sting"',  'the raelets/ray charles orchestra',
                      'zayn / taylor swift']

sorted_slash = []
main_appears_elsewhere_slash = []
wrong_check_slash = []

for artist in billboard['artist_clean']:
    main_artist = artist.split('/')[0] 
    if '/' in artist:
        
        if artist in meaningful_slash:
            sorted_slash.append(artist)
            
        elif artist in after_slash_real_artist:
            sorted_slash.append(artist.split('/')[1])
            
        elif artist in actually_featuring:
            sorted_slash.append(artist.split('/')[0])
        
        elif main_artist in billboard.artist_clean.unique():
            sorted_slash.append(main_artist)
        
        else:
            sorted_slash.append(artist)
            
    else:
        sorted_slash.append(artist)   

In [33]:
billboard.artist_clean = sorted_slash

#### for reference: example of checking whether certain artist has genre tag

In [58]:
results = sp.search(q='artist:paul simon', type='artist', limit=1)

for i, t in enumerate(results['artists']['items']):

    print(t['genres'])

['classic rock', 'folk', 'folk rock', 'folk-pop', 'mellow gold', 'permanent wave', 'rock', 'roots rock', 'singer-songwriter', 'soft rock']


### &

In [41]:
actually_feat = ['chris thompson & night', 'paul humphrey & his cool aid chemists', 'jorgen ingmann & his guitar'
'mos def & pharoahe monch', "al greene & the soul mate's", 'mantovani & his orch.', 'cliff nobles & co.',
'martin solveig & dragonette', 'mc nas-d & dj fred', 'doc box & b. fresh', 'buster poindexter & his banshees of blue',
'gesaffelstein & the weeknd', 'adam levine & javier colon', 'the swell season glen hansard & marketa irglova',
'ike & tina turner & the ikettes', 'anuel aa & romeo santos', 'a r rahman & the pussycat dolls', 
'redhead kingpin & the f.b.i.', 'jonell & method man', 'lucas grabeel & ashley tisdale', 'anuel aa & karol g',
'edward byrnes & connie stevens', 'dillon francis & dj snake', 'dj sammy & yanou', 'mustard & migos',
'doctor dre & ed lover', 'mos def & kweli are black star', "opetaia foa'i & lin-manuel miranda"]

miscellaneous_ampersand = {'gaylord & holiday':'gaylords', 'sylvia r&b':'sylvia', 'dionne & friends':
'dionne warwick', 'dionne':'dionne warwick', 'deliverance/eric weissberg & steve mandell': 
'deliverance', 'lillywood & robin schulz': 'lilly wood', 'delaney & bonnie & friends/eric clapton'
:'delaney & bonnie', 'paul & linda mccartney':'paul mccartney', 'jon & robin and the in crowd': 
'jon & robin', 'keala settle & the greatest showman ensemble':'the greatest showman ensemble', 
'delaney & bonnie & friends': 'delaney & bonnie', 'a-trak + milo & otis':'a-trak',
                               'sonny & cher': 'cher'}

main_appears_elsewhere_wrong = ['Rufus & Carla']

sorted_ampersand = []

for artist in billboard['artist_clean']:
    main_artist = artist.split(' & ')[0] 
    if ' & ' in artist:
        
        if artist in list(miscellaneous_ampersand.keys()):
            sorted_ampersand.append(miscellaneous_ampersand[artist])
        
        elif artist in actually_feat:
            sorted_ampersand.append(main_artist)
            
        elif artist in main_appears_elsewhere_wrong:
            sorted_ampersand.append(artist)
            
        elif main_artist in billboard.artist_clean.unique():
            sorted_ampersand.append(main_artist)
        
        else:
            sorted_ampersand.append(artist)
            
    else:
        sorted_ampersand.append(artist) 

In [42]:
billboard.artist_clean = sorted_ampersand

### and

In [44]:
actually_feat_and = [ 'grand master melle mel and the furious five',  'the used and my chemical romance']

sorted_and = []

for artist in billboard['artist_clean']:
    main_artist = artist.split(' and ')[0] 
    if ' and ' in artist:
            
        if artist in actually_feat_and:
            sorted_and.append(artist.split(' & ')[0])
        
        elif main_artist in billboard.artist_clean.unique() and 'love' not in main_artist:
            sorted_and.append(main_artist)
        
        else:
            sorted_and.append(artist)
            
    else:
        sorted_and.append(artist) 
        

In [45]:
billboard.artist_clean = sorted_ampersand

#### check special characters

In [380]:
for x in billboard[billboard.artist_clean.str.contains(pat = '[^\w\s.]+', regex = True)].artist_clean.unique()[:8]:  
    print(x)

yarbrough & peoples
huey "piano" smith
49-ers
bone thugs-n-harmony
jay-z
ke$ha
p!nk
eagle-eye cherry


#### miscelleanous incorrect entries 

In [51]:
billboard.artist_clean.replace({'m:g':'m.g', 'stars on 54: ultra nate':'ultra nate',
                               'delaney & bonnie & friends/eric clapton':'delaney & bonnie',
                                "fu-schnickens w/ shaquille o'neal":'fu-schnickens',
                                'esther phillips "little esther"':'esther phillips',
                                'Jackson 5': 'The Jackson 5', 'Beatles': 'The Beatles'
                                'wham! u.k.':'wham'},inplace=True)

# *

### save billboard

In [180]:
billboard.to_csv('billboard_final', encoding='utf-8', index=False)

### load billboard

In [181]:
billboard = pd.read_csv('billboard_final')

# part 2

# get artist genre/artist popularity spotipi

### authenticate

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

client_id = 'c70a3fc87cf541198f93adf301838ee5'
client_secret = '744cfea7c14448ed98d9ac8c80052241'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

### scrape


In [312]:
# create empty lists where the results are going to be stored

artist_name = []
genre = []
popularity = []
followers = []
artist_uri = []
billboard_artist = []

import time
start_time = time.time()
request_count = 0

for artist in billboard.artist_clean.unique():
    
    billboard_artist.append(artist)
    
    results = sp.search(q='artist:' + artist, type='artist', limit=1)
    
    if len(results['artists']['items']):

        for i, t in enumerate(results['artists']['items']): 

            try:
                artist_name.append(t['name'])
            except:
                artist_name.append(np.nan)

            try:
                genre.append(t['genres'])
            except:
                genre.append(np.nan)

            try:
                popularity.append(t['popularity'])
            except:
                popularity.append(np.nan)

            try:
                followers.append(t['followers']['total'])
            except:
                followers.append(np.nan)

            try:
                artist_uri.append(t['uri'])
            except:
                artist_uri.append(np.nan)
                
    else:
        artist_name.append(np.nan)
        genre.append(np.nan)
        popularity.append(np.nan)
        followers.append(np.nan)
        artist_uri.append(np.nan)

        request_count+=1
        if request_count % 50 == 0:
            print(str(request_count) + " artists completed")
            print('elapsed: {} seconds'.format(time.time() - start_time))


retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
50 artists completed
elapsed: 118.29415225982666 seconds
retrying ...1secs
retrying ...1secs
retrying ...1secs
100 artists completed
elapsed: 210.17841815948486 seconds
retrying ...2secs
retrying ...1secs
150 artists completed
elapsed: 275.2615079879761 seconds
200 artists completed
elapsed: 327.2434952259064 seconds
retrying ...2secs
retrying ...1secs
250 artists completed
elapsed: 381.80410408973694 seconds
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
300 artists completed
elapsed: 436.672425031662 seconds
retrying ...1secs
350 artists completed
elapsed: 481.08609414100647 seconds
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
400 artists completed
elapsed: 518.356602191925 seconds
450 artists completed
elapsed: 547.5465829372406 seconds
retrying ...1secs
retrying ...1secs
500 artists comple

### create artist_info dataframe

In [319]:
artist_info = pd.DataFrame({'spotify_artist':artist_name, 'billboard_artist':billboard_artist , 
                            'artist_uri':artist_uri, 'genre' : genre, 'artist_popularity' : popularity, 
                            'followers' : followers})

In [166]:
artist_info.head()

,spotify_artist,billboard_artist,artist_uri,genre,artist_popularity,followers
0,Jane Child,jane child,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],36.0,4884.0
1,Jason Derulo,jason derulo,spotify:artist:07YZf4WDAMNwqr4jfgOZ8y,"['dance pop', 'pop', 'pop rap', 'post-teen pop...",83.0,7195281.0
2,Maroon 5,maroon 5,spotify:artist:04gDigrS5kc9YWfZHwBETP,"['pop', 'pop rock']",93.0,20940000.0
3,Bee Gees,bee gees,spotify:artist:1LZEQNv7sE11VDY3SdxQeN,"['adult standards', 'disco', 'mellow gold', 's...",78.0,2636380.0
4,Gloria Estefan,gloria estefan,spotify:artist:5IFCkqu9J6xdWeYMk5I889,"['dance pop', 'latin pop', 'new wave pop', 'so...",68.0,347934.0


### save 

In [321]:
artist_info.to_csv('artist_info_final', encoding='utf-8', index=False)

### load artist_info

In [158]:
artist_info = pd.read_csv('artist_info_final')

# *

# part 3

# merge to get billboard + artist_info

In [682]:
billboard_merged = pd.merge(billboard.drop('artist', axis=1) , artist_info , 
                                   how='inner', left_on='artist_clean', right_on='billboard_artist')
billboard_merged.head(2)

,track_name,WeekID,Week Position,Instance,Weeks on Chart,Peak Position,artist_clean,spotify_artist,billboard_artist,artist_uri,genre,artist_popularity,followers
0,don't wanna fall in love,2/10/1990,75,1,1,75,jane child,Jane Child,jane child,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],36.0,4884.0
1,don't wanna fall in love,2/17/1990,53,1,2,53,jane child,Jane Child,jane child,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],36.0,4884.0


## check that merge is correct

#### check that billboard & new dataframe have same length

In [340]:
billboard_merged.shape[0] == billboard.shape[0]

True

### check that the merged columns are the same

In [511]:
incorrect = billboard_merged.loc[(billboard_merged['artist_clean'] != [x.lower() for x in 
billboard_merged['spotify_artist']])].drop_duplicates(subset='artist_clean', 
keep='first')[['artist_clean', 'spotify_artist']].head(10)
print(incorrect.to_string())

                        artist_clean                   spotify_artist
2996                     john mayall  John Mayall & The Bluesbreakers
3111                 vincent edwards               J. Vincent Edwards
4833                          linear                      Linear Labs
5198                           touch                 A Touch Of Class
11221  perez prado and his orchestra      Perez Prado & his Orchestra
17590                          jay-z                            JAY Z
20906                    celine dion                      Céline Dion
21514                          ke$ha                      Ha Ke Kgomo
25208                     puff daddy                Mase & Puff Daddy
26577                        beyonce                          Beyoncé


#### create a dictionary of entries where billboard doesnt match up with spotify results

In [436]:
incorrect_dict = {}
for artist_name in incorrect.artist_clean:
    incorrect_dict[artist_name]=  list(incorrect.groupby('artist_clean')['track_name'].get_group(artist_name).values)

#### scrape again including track name to try get more accurate scrape

In [479]:
billboard_artist = []
improved_artist = []

for artist_name in incorrect_dict:
    for track in set(incorrect_dict[artist_name]):
        
        billboard_artist.append(artist_name)
        
        results = sp.search(q='artist:' + artist_name + ' track:' + track, type='track', limit=1)

        if len(results['tracks']['items']):

            for i, t in enumerate(results['tracks']['items']):

                improved_artist.append(t['album']['artists'][0]['name'])
                
        else:
            improved_artist.append(np.nan)           

retrying ...1secs
retrying ...1secs


#### put into dataframe

In [499]:
improved = pd.DataFrame({'improved_artist':improved_artist, 'billboard_artist':billboard_artist })

In [535]:
improved.head(2)

,improved_artist,billboard_artist
0,NaN,john mayall
1,NaN,vincent edwards


In [529]:
# not nan
improved_nn = improved[pd.notnull(improved.improved_artist)]

In [536]:
add_to_original = improved_nn.loc[(improved_nn.billboard_artist == [x.lower() for x in improved_nn.improved_artist])]
add_to_original 

,improved_artist,billboard_artist
24,Blaque,blaque
27,Prince,prince
29,Free,free
30,Holly Knight,holly knight
32,The Originals,the originals
...,...,...
1135,Notorious,notorious
1137,Tag,tag
1149,Lake,lake
1156,the Triplets,the triplets


In [564]:
check_these = improved_nn.loc[(improved_nn.billboard_artist != [x.lower() 
                            for x in improved_nn.improved_artist])][check_these.improved_artist != 'Various Artists' ]
check_these.head()

,improved_artist,billboard_artist
6,Céline Dion,celine dion
9,Beyoncé,beyonce
10,Janet Jackson,janet
13,John Mellencamp,john cougar mellencamp
15,Herb Alpert & The Tijuana Brass,herb alpert


In [180]:
#https://github.com/seatgeek/fuzzywuzzy

In [566]:
to_drop = [19, 25, 47, 52, 57, 59, 70, 81, 90, 99, 143, 203, 215, 246, 264, 269, 318, 433, 518, 558, 561, 593, 598, 
646, 665, 719, 795, 800, 815, 881, 908, 910, 934, 996, 997, 1042, 1052, 1054, 1094, 1103, 1104, 1152, 1153, 1155 ]

In [567]:
check_these.drop(df.index[to_drop], inplace=True)

In [571]:
add_to_original.append(check_these).head()

,improved_artist,billboard_artist
24,Blaque,blaque
27,Prince,prince
29,Free,free
30,Holly Knight,holly knight
32,The Originals,the originals


In [ ]:
for x in num, artist in enumerate(add_to_original.billboard_artist):
    billboard_merged.spotify_artist.str.replace()
    df['col']= df['col'].str.replace('a', 'b')

In [683]:
billboard_merged = billboard_merged[~billboard_merged.billboard_artist.isin(list(add_to_original.billboard_artist))]

## null values 

In [354]:
billboard_merged.to_csv('billboard_artist_merge_including_nans', encoding='utf-8', index=False)

In [684]:
print(billboard_merged.shape)
billboard_merged.dropna(inplace=True)
print(billboard_merged.shape)

(311213, 13)
(301721, 13)


In [685]:
billboard_merged.isnull().sum()

track_name           0
WeekID               0
Week Position        0
Instance             0
Weeks on Chart       0
Peak Position        0
artist_clean         0
spotify_artist       0
billboard_artist     0
artist_uri           0
genre                0
artist_popularity    0
followers            0
dtype: int64

#### duplicates check

In [678]:
print(len(artist_info[artist_info.duplicated(keep='first')]))

0


#### nice presentation

In [688]:
billboard_merged.columns = [c.lower().replace(' ', '_') for c in billboard_merged.columns]

In [690]:
billboard_merged = billboard_merged [['spotify_artist', 'track_name', 'billboard_artist', 'artist_uri', 'genre', 'artist_popularity',
                    'followers', 'weekid', 'week_position', 'instance', 'weeks_on_chart', 'peak_position']]

In [681]:
billboard_merged.head(2)

,spotify_artist,track_name,artist_uri,genre,artist_popularity,followers,weekid,week_position,instance,weeks_on_chart,peak_position
0,Jane Child,don't wanna fall in love,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],36.0,4884.0,2/10/1990,75,1,1,75
1,Jane Child,don't wanna fall in love,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],36.0,4884.0,2/17/1990,53,1,2,53


### save 

In [691]:
billboard_merged.to_csv('billboard_merged_final', encoding='utf-8', index=False)

### load

In [204]:
billboard_merged = pd.read_csv('billboard_merged_final')

In [205]:
billboard_merged .shape

(301721, 12)

########################################################################################################################

# part 4 - individual track info

## clean track name

In [598]:
for x in billboard_merged[billboard_merged.track_name.str.contains(pat = '[^\w\s.]+', regex = True)].track_name.unique()[:10]:  
    print(x)

don't wanna fall in love
don't wanna go home
ridin' solo
if it ain't love
don't wanna know
won't go home without you
this summer's gonna hurt...
don't wanna live inside myself
stayin' alive
new york mining disaster 1941 (have you seen my wife, mr. jones)


In [599]:
billboard_merged.track_name.replace({'new york mining disaster 1941 (have you seen my wife, mr. jones)':'new york mining disaster 1941', 
    "ev'rybody's twistin'": "everybody's twistin'"},inplace=True)

## track id (to get musical components)/ track popularity

In [600]:
artist_dict={}
for artist_name in billboard_merged.spotify_artist.unique():
    artist_dict[artist_name]=list(billboard_merged.groupby('spotify_artist')['track_name'
                                                                            ].get_group(artist_name).values)

In [237]:
for artist_name in artist_dict:
    for track_name in set(artist_dict[artist_name]):
            print(artist_name, track_name )

Jane Child Don't Wanna Fall In Love
Jane Child Welcome To The Real World
Jason Derulo If It Ain't Love
Jason Derulo It Girl
Jason Derulo Don't Wanna Go Home
Jason Derulo Fight For You
Jason Derulo The Other Side
Jason Derulo Want To Want Me
Jason Derulo Undefeated
Jason Derulo Trumpets
Jason Derulo Get Ugly
Jason Derulo In My Head
Jason Derulo Whatcha Say
Jason Derulo Marry Me
Jason Derulo Cheyenne
Jason Derulo Ridin' Solo
Jason Derulo What If
Bee Gees I.O.I.O.
Bee Gees Words
Bee Gees First Of May
Bee Gees Still Waters (Run Deep)
Bee Gees I've Gotta Get A Message To You
Bee Gees Alive
Bee Gees Paying The Price Of Love
Bee Gees Nights On Broadway
Bee Gees You Win Again
Bee Gees Living Eyes
Bee Gees Night Fever
Bee Gees Fanny (Be Tender With My Love)
Bee Gees If Only I Had My Mind On Something Else
Bee Gees Love So Right
Bee Gees Don't Forget To Remember
Bee Gees Stayin' Alive
Bee Gees Don't Wanna Live Inside Myself
Bee Gees Tragedy
Bee Gees To Love Somebody
Bee Gees Tomorrow Tomorrow
Be

Lady Gaga Venus
Lady Gaga Dope
Lady Gaga Applause
Lady Gaga Born This Way
Lady Gaga Til It Happens To You
Lady Gaga You And I
Lady Gaga A-YO
Lady Gaga Judas
Lady Gaga Alejandro
Lady Gaga Poker Face
Katy Perry Never Really Over
Katy Perry Circle The Drain
Katy Perry The One That Got Away
Katy Perry Unconditionally
Katy Perry Wide Awake
Katy Perry Walking On Air
Katy Perry Hot N Cold
Katy Perry Thinking Of You
Katy Perry This Is How We Do
Katy Perry Birthday
Katy Perry Rise
Katy Perry Firework
Katy Perry Teenage Dream
Katy Perry Part Of Me
Katy Perry Cozy Little Christmas
Katy Perry Not Like The Movies
Katy Perry Waking Up In Vegas
Katy Perry Last Friday Night (T.G.I.F.)
Katy Perry Roar
Katy Perry I Kissed A Girl
Taylor Swift Breathless
Taylor Swift White Horse
Taylor Swift Fifteen
Taylor Swift The Other Side Of The Door
Taylor Swift I Knew You Were Trouble.
Taylor Swift Teardrops On My Guitar
Taylor Swift Jump Then Fall
Taylor Swift Crazier
Taylor Swift You're Not Sorry
Taylor Swift Sta

The Weeknd Die For You
The Weeknd True Colors
The Weeknd Can't Feel My Face
The Weeknd Wicked Games
The Weeknd Often
The Weeknd In The Night
The Weeknd Rockin'
The Weeknd Attention
The Weeknd Real Life
The Weeknd Where You Belong
The Weeknd Ordinary Life
The Weeknd Call Out My Name
The Weeknd A Lonely Night
The Weeknd Secrets
The Weeknd Six Feet Under
The Weeknd Reminder
The Weeknd Privilege
Travis Scott Yosemite
Travis Scott Goosebumps
Travis Scott Stop Trying To Be God
Travis Scott Skeletons
Travis Scott NC-17
Travis Scott Carousel
Travis Scott Beibs In The Trap
Travis Scott Wake Up
Travis Scott Who? What!
Travis Scott Coffee Bean
Travis Scott R.I.P Screw
Travis Scott Astrothunder
Travis Scott 5% Tint
Travis Scott No Bystanders
Travis Scott Butterfly Effect
Travis Scott Houstonfornication
Travis Scott Antidote
Travis Scott Sicko Mode
Travis Scott Can't Say
Travis Scott Stargazing
Spice Girls Say You'll Be There
Spice Girls Wannabe
Spice Girls Stop
Spice Girls Spice Up Your Life
Spice

Lil Wayne Hustler Musik
Lil Wayne President Carter
Lil Wayne Dedicate
Lil Wayne Uproar
Lil Wayne Go D.J.
Lil Wayne 3 Peat
Lil Wayne Let It All Work Out
Lil Wayne Demon
Lil Wayne I Am Not A Human Being
Lil Wayne How To Love
Lil Wayne Blunt Blowin
Lil Wayne Hittas
K-Ci & JoJo You Bring Me Up
K-Ci & JoJo Life
K-Ci & JoJo Tell Me It's Real
K-Ci & JoJo Crazy
K-Ci & JoJo Last Night's Letter
K-Ci & JoJo All My Life
K-Ci & JoJo How Could You (From "Bulletproof")
The Danleers One Summer Night
Rihanna Only Girl (In The World)
Rihanna Shut Up And Drive
Rihanna Sex With Me
Rihanna Don't Stop The Music
Rihanna American Oxygen
Rihanna B**** Better Have My Money
Rihanna California King Bed
Rihanna SOS
Rihanna What Now
Rihanna Cheers (Drink To That)
Rihanna Rude Boy
Rihanna Take A Bow
Rihanna Love On The Brain
Rihanna Needed Me
Rihanna Pour It Up
Rihanna Pon de Replay
Rihanna Russian Roulette
Rihanna Diamonds
Rihanna Disturbia
Rihanna You Da One
Rihanna Kiss It Better
Rihanna Rehab
Rihanna Man Down
Ri

Def Leppard Foolin'
Def Leppard Rocket
Def Leppard Stand Up (Kick Love Into Motion)
Def Leppard Love Bites
Def Leppard Pour Some Sugar On Me
Def Leppard When Love & Hate Collide
Def Leppard Bringin' On The Heartbreak
Def Leppard Hysteria
Def Leppard Animal
Def Leppard Photograph
Def Leppard Have You Ever Needed Someone So Bad
Def Leppard Tonight
Def Leppard Armageddon It
Def Leppard Rock Of Ages
Ugly Kid Joe Everything About You
Ugly Kid Joe Cat's In The Cradle
2Pac Changes
2Pac Thugz Mansion
2Pac I Wonder If Heaven Got A Ghetto
2Pac Keep Ya Head Up
2Pac I Get Around
2Pac Until The End Of Time
2Pac Temptations
2Pac Dear Mama/Old School
2Pac So Many Tears
Roy Orbison Twinkle Toes
Roy Orbison Up Town
Roy Orbison It's Over
Roy Orbison Running Scared
Roy Orbison Let The Good Times Roll
Roy Orbison In Dreams
Roy Orbison Too Soon To Know
Roy Orbison Communication Breakdown
Roy Orbison Falling
Roy Orbison I'm Hurtin'
Roy Orbison (Say) You're My Girl
Roy Orbison Candy Man
Roy Orbison Mean Woma

...

DJ Luke Nasty Might Be
Ferras Hollywood's Not America
Kreayshawn Gucci Gucci
Taja Sevelle Love Is Contagious
Inner City Whatcha Gonna Do With My Lovin'
Inner City Good Life
Keith Murray Get Lifted
Keith Murray The Most Beautifullest Thing In This World
The Replacements I'll Be You
Mike Jones Next To You
Mike Jones Back Then
Mike Jones Mr. Jones
A Few Good Men Have I Never
MC Serch Here It Comes
Nico & Vinz In Your Arms
Nico & Vinz Am I Wrong
Carl Douglas Kung Fu Fighting
Carl Douglas Dance The Kung Fu
Donna Lewis I Love You Always Forever
Donna Lewis Without Love
The Floaters Float On
Eddy Grant Electric Avenue
Eddy Grant I Don't Wanna Dance
Eddy Grant Romancing The Stone
Yung Joc It's Goin' Down
James Ingram There's No Easy Way
James Ingram I Don't Have The Heart
Elvin Bishop Fooled Around And Fell In Love
Elvin Bishop Sure Feels Good
Elvin Bishop Travelin' Shoes
Elvin Bishop Struttin' My Stuff
Deee-Lite Groove Is In The Heart
Deee-Lite Power Of Love
Suzanne Vega Solitude Standing
Suz

Rob Jungklas Make It Mean Something
Glen Burtnick Follow You
B Angie B I Don't Want To Lose Your Love
A Rocket To The Moon Like We Used To
Roscoe Dash Good Good Night
Cody Johnson On My Way To You
Don French Lonely Saturday Night
The Capris Limbo
The Capris Where I Fell In Love
The Capris There's A Moon Out Tonight
The Capris Girl In My Dreams
Troy Shondell This Time
Troy Shondell Tears From An Angel
Troy Shondell Island In The Sky
Tommy Boyce I'll Remember Carol
The Butterflys Good Night Baby
Bud Shank Michelle
Ruby Andrews Hey Boy Take A Chance On Love
Ruby Andrews You Made A Believer (Out Of Me)
Ruby Andrews Casonova (Your Playing Days Are Over)
The Jaggerz The Rapper
The Jaggerz What A Bummer
The Jaggerz I Call My Baby Candy
Jean Beauvoir Feel The Heat
The 4 Of Us Drag My Bad Name Down
Anastacia I'm Outta Love
Eddie Vedder My City Of Ruins (Live From The Kennedy Center Honors)
Barrett Baber I'd Just Love To Lay You Down
Carl Smith Ten Thousand Drums
Carl Smith Guess I've Been Aroun

Frankie Karl & The Dreams Don't Be Afraid (Do As I Say)
Porter Wagoner The Carroll County Accident
Roy Clark Yesterday, When I Was Young
Roy Clark Come Live With Me
Roy Clark Thank God And Greyhound
Roy Clark Tips Of My Fingers
Roy Clark Then She's A Lover
Roy Clark Somewhere Between Love And Tomorrow
Danny Holien Colorado
The Temprees Dedicated To The One I Love
Holland-Dozier Why Can't We Be Lovers
Holland-Dozier Don't Leave Me Starvin' For Your Love (Part 1)
Houston Person Disco Sax/for The Love Of You
Donna McDaniel Save Me
La Flavour Only The Lonely (Have A Reason To Be Sad)
Snuff Bad, Bad Billy
Jenny Burton Remember What You Like
Alex Brown (Come On) Shout
Far Corporation Stairway To Heaven
Newcity Rockers Black Dog
Newcity Rockers Rev It Up
Martha Davis Don't Tell Me The Time
The Fabulous Thunderbirds Tuff Enuff
The Fabulous Thunderbirds Wrap It Up
The Fabulous Thunderbirds Powerful Stuff (From "Cocktail")
The Fabulous Thunderbirds Stand Back
Sam Brown Stop
Jimmy Ryser Same Old 

Red River Dave There's A Star Spangled Banner Waving #2 (The Ballad Of Francis Powers)
Dorothy Norwood There's Got To Be Rain In Your Life (To Appreciate The Sunshine)
Bobby Marchan There's Something On Your Mind (Part 2)
Rico Love They Don't Know
Napoleon XIV They're Coming To Take Me Away, Ha-Haaa!
Cheryl Ladd Think It Over
Grandmaster Slice Thinking Of You
B.G. The Prince Of Rap This Beat Is Hot
Gene Redding This Heart
S.O.A.P. This Is How We Party
Melba Moore You Stepped Into My Life
Melba Moore This Is It
Laura Enea This Is The Last Time
Bob Geldof This Is The World Calling
Don Goodwin This Is Your Song
Bill LaBounty This Night Won't Last Forever
Purple Reign This Old Man
Mina This World We Love In (Il Cielo In Una Stanza)
Blueface Thotiana
Roy Drusky Three Hearts In A Tangle
Green Jelly Three Little Pigs
Ruby Wright Three Stars
DragonForce Through The Fire And Flames
Tanya Blount Through The Rain
L.V. Throw Your Hands Up
Chi Coltrane Thunder And Lightning
Fruit De La Passion Tic 

Homer & Jethro nan
Flash Cadillac & The Continental Kids nan
Blood Coven nan
The Hit Parade nan
The Natural Four nan
Hot Flash Heat Wave nan
The Facts of Life nan
Kix nan
T-Ray Da Don nan
Gang Situ and Gary Schwantes nan
The DIllman Band nan
Richard Dimples Fields nan
Jimmy Boyd; Accompanied by Norman Luboff nan
Elvira, Mistress of the Dark nan
Rtz nan
Billy Preston And Syreeta nan
Missy Misdemeanor Elliott nan
GZA (Genius) nan
Miz Korona nan
Forest for the Trees nan
The Ruby Tuesdays nan
The Boston Pops Orchestra & Arthur Fiedler nan
Classics IV - Featuring Dennis Yost nan
Georgie Young And The Rockin' Bocs nan
Travis Matte And The Zydeco Kingpins nan
Sérgio Mendes nan
Don Julian & The Larks nan
The G Clefs nan
Edmundo Ros & His Orchestra nan
Psycho Boys Club nan
Snøw nan
Joiner, Arkansas Jr. High School Band nan
Richard Hayman & His Orchestra nan
Harold Melvin & The Blue Notes nan
Brighter Side of Darkness nan
Latoya Jackson nan
Natalie Hemby nan
Hotlegs© nan
Derez De’Shon nan
Mono K

In [605]:
original_track = []
original_artist = []

track_name = []
artist_name = []
track_popularity = []
track_uri = []

import time
start_time = time.time()
request_count = 0
    
for artist in artist_dict:
    for track in set(artist_dict[artist]):
        
        original_track.append(track)
        original_artist.append(artist)
        
        track_results = sp.search(q='artist:' + artist + ' track:' + track, type='track', limit=1)
        
        if len(track_results['tracks']['items']):

            for i, t in enumerate(track_results['tracks']['items']):

                try:
                    track_name.append(t['name'])
                except:
                    track_name.append(np.nan)

                try:
                    artist_name.append(t['album']['artists'][0]['name'])
                except:
                    artist_name.append(np.nan)

                try:
                    track_popularity.append(t['popularity'])
                except:
                    track_popularity.append(np.nan)

                try:
                    track_uri.append(t['uri'])
                except:
                    track_uri.append(np.nan)
                    
        else:
            track_name.append(np.nan)
            artist_name.append(np.nan)
            track_popularity.append(np.nan)
            track_uri.append(np.nan)
                    
        request_count+=1
        if request_count % 50 == 0:
            print(str(request_count) + " track completed")
            print('elapsed: {} seconds'.format(time.time() - start_time))


50 track completed
elapsed: 6.420135259628296 seconds
100 track completed
elapsed: 12.938940048217773 seconds
150 track completed
elapsed: 19.71945810317993 seconds
200 track completed
elapsed: 26.20376491546631 seconds
250 track completed
elapsed: 32.74305510520935 seconds
300 track completed
elapsed: 39.20877718925476 seconds
350 track completed
elapsed: 45.64628601074219 seconds
400 track completed
elapsed: 52.11659216880798 seconds
450 track completed
elapsed: 58.57183003425598 seconds
500 track completed
elapsed: 64.96647596359253 seconds
550 track completed
elapsed: 71.35227012634277 seconds
600 track completed
elapsed: 78.12748789787292 seconds
650 track completed
elapsed: 84.7105278968811 seconds
700 track completed
elapsed: 91.21583604812622 seconds
750 track completed
elapsed: 97.77310824394226 seconds
800 track completed
elapsed: 104.63830900192261 seconds
850 track completed
elapsed: 111.41532611846924 seconds
900 track completed
elapsed: 117.85985922813416 seconds
950 trac

7400 track completed
elapsed: 944.9594962596893 seconds
7450 track completed
elapsed: 951.4359929561615 seconds
7500 track completed
elapsed: 957.9702272415161 seconds
7550 track completed
elapsed: 964.300684928894 seconds
7600 track completed
elapsed: 970.4894251823425 seconds
7650 track completed
elapsed: 976.730945110321 seconds
7700 track completed
elapsed: 983.1170520782471 seconds
7750 track completed
elapsed: 989.2207992076874 seconds
7800 track completed
elapsed: 995.4864649772644 seconds
7850 track completed
elapsed: 1001.7108352184296 seconds
7900 track completed
elapsed: 1008.2023038864136 seconds
7950 track completed
elapsed: 1014.399943113327 seconds
8000 track completed
elapsed: 1020.675873041153 seconds
8050 track completed
elapsed: 1027.29722905159 seconds
8100 track completed
elapsed: 1033.9121010303497 seconds
8150 track completed
elapsed: 1040.3101439476013 seconds
8200 track completed
elapsed: 1046.5554139614105 seconds
8250 track completed
elapsed: 1052.71701025962

14550 track completed
elapsed: 1853.3866732120514 seconds
14600 track completed
elapsed: 1859.6045832633972 seconds
14650 track completed
elapsed: 1865.8025970458984 seconds
14700 track completed
elapsed: 1872.4471080303192 seconds
14750 track completed
elapsed: 1878.8160109519958 seconds
14800 track completed
elapsed: 1885.0908410549164 seconds
14850 track completed
elapsed: 1891.5622231960297 seconds
14900 track completed
elapsed: 1897.8495359420776 seconds
14950 track completed
elapsed: 1904.47176527977 seconds
15000 track completed
elapsed: 1910.9503300189972 seconds
15050 track completed
elapsed: 1917.2363150119781 seconds
15100 track completed
elapsed: 1923.608915090561 seconds
15150 track completed
elapsed: 1929.8495671749115 seconds
15200 track completed
elapsed: 1936.2663960456848 seconds
15250 track completed
elapsed: 1942.7769062519073 seconds
15300 track completed
elapsed: 1949.0430200099945 seconds
15350 track completed
elapsed: 1955.279895067215 seconds
15400 track comple

21700 track completed
elapsed: 2750.9787061214447 seconds
21750 track completed
elapsed: 2757.168134212494 seconds
21800 track completed
elapsed: 2763.3408250808716 seconds
21850 track completed
elapsed: 2769.3658401966095 seconds
21900 track completed
elapsed: 2775.4307622909546 seconds
21950 track completed
elapsed: 2781.582556247711 seconds
22000 track completed
elapsed: 2787.736088991165 seconds
22050 track completed
elapsed: 2794.0338559150696 seconds
22100 track completed
elapsed: 2800.2622051239014 seconds
22150 track completed
elapsed: 2806.3804700374603 seconds
22200 track completed
elapsed: 2812.633429288864 seconds
22250 track completed
elapsed: 2818.7678260803223 seconds
22300 track completed
elapsed: 2825.098916053772 seconds
22350 track completed
elapsed: 2831.861974000931 seconds
22400 track completed
elapsed: 2838.456153154373 seconds
22450 track completed
elapsed: 2844.8397040367126 seconds
22500 track completed
elapsed: 2851.440903186798 seconds
22550 track completed


#### track name / track popularity / track ID dataframe

In [608]:
track_info = pd.DataFrame({'original_track':original_track, 'original_artist':original_artist, 'artistor':artist_name, 
                           'track_name':track_name, 'track_popularity':track_popularity, 'track_uri': track_uri })

In [609]:
track_info.head()

,original_track,original_artist,artist,track_name,track_popularity,track_uri
0,don't wanna fall in love,Jane Child,NaN,NaN,NaN,NaN
1,welcome to the real world,Jane Child,Jane Child,Welcome to the Real World,19.0,spotify:track:37o0UBywazf2Xcy3297Que
2,it girl,Jason Derulo,Jason Derulo,It Girl,72.0,spotify:track:4fINc8dnfcz7AdhFYVA4i7
3,if it ain't love,Jason Derulo,Jason Derulo,If It Ain't Love,62.0,spotify:track:5YHR8wBR5jXfGtP766NiAT
4,in my head,Jason Derulo,Jason Derulo,In My Head,65.0,spotify:track:5BoIP8Eha5hwmRVURkC2Us


In [610]:
len(track_info.artist.unique())

4511

In [611]:
len(track_info.track_name.unique())

18937

In [612]:
track_info.isnull().sum()

original_track         0
original_artist        0
artist              5419
track_name          5419
track_popularity    5419
track_uri           5419
dtype: int64

In [613]:
print(track_info.shape)
track_info.dropna(inplace=True)
print(track_info.shape)

(26517, 6)
(21098, 6)


In [615]:
print(len(track_info[track_info.duplicated(keep='first')]))

0


# *

## musical components

In [616]:
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
tempo = []
valence = []
track_uri = []

start_time = time.time()
request_count = 0   

for track in track_info.track_uri:

    feature_results = sp.audio_features(track)
    
    for i, t in enumerate(feature_results):
        
        try:
            track_uri.append(t['uri'])
        except:
            track_uri.append(np.nan)
        
        try:
            acousticness.append(t['acousticness'])
        except:
            acousticness.append(np.nan)
 
        try:
            danceability.append(t['danceability'])
        except:
            danceability.append(np.nan)
       
        try:
            energy.append(t['energy'])
        except:
            energy.append(np.nan)
        
        try:
            instrumentalness.append(t['instrumentalness'])
        except:
            instrumentalness.append(np.nan)
            
        try:
            liveness.append(t['liveness'])
        except:
            liveness.append(np.nan)
            
        try:
            loudness.append(t['loudness'])
        except:
            loudness.append(np.nan)
             
        try:
            speechiness.append(t['speechiness'])
        except:
            speechiness.append(np.nan)
             
        try:
            tempo.append(t['tempo'])
        except:
            tempo.append(np.nan)
        
        try:
            valence.append(t['valence'])
        except:
            valence.append(np.nan)
            
        try:
            track_uri.append(t['uri'])
        except:
            track_uri.append(np.nan)

        try:
            duration.append(t['duration_ms'])
        except:
            duration.append(np.nan)

        try:
            key.append(t['key'])
        except:
            key.append(np.nan)

        try:
            mode.append(t['mode'])
        except:
            mode.append(np.nan)

        try:
            time_signature.append(t['time_signature'])
        except:
            time_signature.append(np.nan)
        
        
    request_count+=1
    if request_count % 100 == 0:
        print(str(request_count) + " musical components completed")
        print('elapsed: {} seconds'.format(time.time() - start_time))


100 musical components completed
elapsed: 10.937906265258789 seconds
200 musical components completed
elapsed: 21.875027179718018 seconds
300 musical components completed
elapsed: 32.992112159729004 seconds
400 musical components completed
elapsed: 44.052175998687744 seconds
500 musical components completed
elapsed: 56.15140914916992 seconds
600 musical components completed
elapsed: 67.35504817962646 seconds
700 musical components completed
elapsed: 77.85849809646606 seconds
800 musical components completed
elapsed: 88.66470313072205 seconds
900 musical components completed
elapsed: 99.71672105789185 seconds
1000 musical components completed
elapsed: 110.30009698867798 seconds
1100 musical components completed
elapsed: 120.98076009750366 seconds
1200 musical components completed
elapsed: 131.85294103622437 seconds
1300 musical components completed
elapsed: 142.84583401679993 seconds
1400 musical components completed
elapsed: 153.8482151031494 seconds
1500 musical components completed
e

12000 musical components completed
elapsed: 1343.935308933258 seconds
12100 musical components completed
elapsed: 1354.9921441078186 seconds
12200 musical components completed
elapsed: 1365.8898811340332 seconds
12300 musical components completed
elapsed: 1376.6842970848083 seconds
12400 musical components completed
elapsed: 1387.700317144394 seconds
12500 musical components completed
elapsed: 1399.2637150287628 seconds
12600 musical components completed
elapsed: 1411.3984282016754 seconds
12700 musical components completed
elapsed: 1422.5561032295227 seconds
12800 musical components completed
elapsed: 1433.3711159229279 seconds
12900 musical components completed
elapsed: 1444.3598251342773 seconds
13000 musical components completed
elapsed: 1455.4050979614258 seconds
13100 musical components completed
elapsed: 1466.6045141220093 seconds
13200 musical components completed
elapsed: 1477.6975359916687 seconds
13300 musical components completed
elapsed: 1488.8963482379913 seconds
13400 mu

### musical components df

In [617]:
musical_components = pd.DataFrame({'track_uri':track_uri, 'acousticness':acousticness, 'danceability':danceability, 
                                   'energy' : energy, 'instrumentalness' : instrumentalness, 'liveness' : liveness, 
                                   'loudness' : loudness, 'speechiness' : speechiness, 
                                    'tempo' : tempo, 'valence' : valence})

new_features = pd.DataFrame({'track_uri':track_uri,'duration':duration, 'key':key, 'mode':mode,
                             'time_signature':time_signature})

In [628]:
print(len(musical_components[musical_components.duplicated(keep='first')]))
musical_components.drop_duplicates(inplace=True)

75


# merge track_all

In [663]:
track_all = pd.merge(track_info, musical_components, left_on="track_uri", right_on="track_uri", how='inner')

In [75]:
track_all.shape

(21094, 15)

In [667]:
print(len(track_all[track_all.duplicated(keep='first')]))

0


In [669]:
track_all.isnull().sum().sum()

0

#### save tracks to csv

In [72]:
track_all.to_csv('track_all_old', encoding='utf-8', index=False)
track_all_new.to_csv('track_all_new', encoding='utf-8', index=False)

#### load tracks from csv

In [170]:
track_all_new = pd.read_csv('track_all_new')

# *

# part 5 - merge track_all + billboard_merged

In [97]:
everything = pd.merge(billboard_merged.drop('billboard_artist', axis=1), track_all_new.drop(['original_artist'],  axis=1), 
          left_on=['spotify_artist', 'track_name'],
          right_on=['artist', 'original_track'], how ='inner')

In [98]:
everything.shape

(236237, 29)

In [99]:
everything.columns

Index(['spotify_artist', 'track_name_x', 'artist_uri', 'genre',
       'artist_popularity', 'followers', 'weekid', 'week_position', 'instance',
       'weeks_on_chart', 'peak_position', 'original_track', 'artist',
       'track_name_y', 'track_popularity', 'track_uri', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'valence', 'duration', 'key', 'mode',
       'time_signature'],
      dtype='object')

#### nice presentation

In [101]:
everything = everything[['spotify_artist', 'track_name_y', 'artist_popularity', 'followers', 'artist_uri', 'genre', 
                        'weekid', 'week_position', 'instance', 'weeks_on_chart','peak_position', 
                        'track_popularity', 'track_uri', 'acousticness', 'danceability',
                        'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
                        'tempo', 'valence', 'duration', 'key', 'mode','time_signature']]
everything.rename(columns={'track_name_y': 'track_name', 'spotify_artist':'artist'}, inplace=True)
everything['weekid'] = pd.to_datetime(everything['weekid'],format='%m/%d/%Y')

#### make sure artists / tracks matched up 

In [183]:
print(len(everything.track_name.unique()))
print(len(everything.artist.unique()))

17693
4323


In [125]:
everything.drop_duplicates(keep='first', inplace=True)

#### save tracks to csv

In [127]:
everything.to_csv('everything_scraped_new', encoding='utf-8', index=False)

#### load tracks from csv

In [3]:
everything = pd.read_csv('everything_scraped_new')

# -----------------------------------------------------------------------------------------

In [6]:
everything.head(4)

,artist,track_name,artist_popularity,followers,artist_uri,genre,weekid,week_position,instance,weeks_on_chart,peak_position,track_popularity,track_uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration,key,mode,time_signature
0,Jane Child,Welcome to the Real World,36.0,4884.0,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],1990-06-02,90,1,1,90,19.0,spotify:track:37o0UBywazf2Xcy3297Que,0.00703,0.625,0.721,0.000001,0.317,-12.512,0.0325,104.651,0.851,299640,8,0,4
1,Jane Child,Welcome to the Real World,36.0,4884.0,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],1990-06-09,77,1,2,77,19.0,spotify:track:37o0UBywazf2Xcy3297Que,0.00703,0.625,0.721,0.000001,0.317,-12.512,0.0325,104.651,0.851,299640,8,0,4
2,Jane Child,Welcome to the Real World,36.0,4884.0,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],1990-06-16,60,1,3,60,19.0,spotify:track:37o0UBywazf2Xcy3297Que,0.00703,0.625,0.721,0.000001,0.317,-12.512,0.0325,104.651,0.851,299640,8,0,4
3,Jane Child,Welcome to the Real World,36.0,4884.0,spotify:artist:5JBif5ahOKFFVuEpHhrp8Y,['new wave pop'],1990-06-23,56,1,4,56,19.0,spotify:track:37o0UBywazf2Xcy3297Que,0.00703,0.625,0.721,0.000001,0.317,-12.512,0.0325,104.651,0.851,299640,8,0,4
